In [1]:
import altair as alt
import altair_stiles as altstiles

alt.themes.register("stiles", altstiles.theme)
alt.themes.enable("stiles")

ThemeRegistry.enable('grid')

In [2]:
import pandas as pd

In [3]:
from oauth2client.service_account import ServiceAccountCredentials
import gspread

In [4]:
from df2gspread import df2gspread as d2g

### datawrapper credentials

In [5]:
from datawrapper import Datawrapper

dw = Datawrapper(
    access_token="FtIwtvFtoGLaRT9a3gjX69PLu4wSuRyKddoOz6SOPw3k9wWyNICMHTkcPhOGCR5Z"
)

### setting up google sheets

In [6]:
scope = ['https://spreadsheets.google.com/feeds']
credentials = ServiceAccountCredentials.from_json_keyfile_name('jupyter-integration-349314-25735d35924b.json', scope)
gc = gspread.authorize(credentials)

### currently set up to get pacific sea surface temperature from a google sheet

In [7]:
spreadsheet_key = "1LXH-tda6l5Mx53bJHUDyDJrAZxifp_0O7F3P47BgtIo"
book = gc.open_by_key(spreadsheet_key)

In [8]:
#Opening the worksheet by using Worksheet ID
workbook = gc.open_by_key(spreadsheet_key)
#Selecting which sheet to pulling the data
sheet = workbook.worksheet('Sheet1')
#Pulling the data and transform it to the data frame
values = sheet.get_all_values()
pacifictemps = pd.DataFrame(values[1:], columns = values[0])

#### how to upload data to google sheets (but need a different spreadsheet key)

In [9]:
#wks_name = "data"
#d2g.upload(wp_sst_dw, spreadsheet_key, wks_name, credentials=credentials, row_names=False)

### annual global SST data

In [10]:
seatemps =pd.read_csv("data/raw/sst.csv")

### wavepower data from study

In [11]:
wavepower_raw=pd.read_csv('data/raw/GOW_CFSR_Global_Wave_Power.txt', sep=' ')

In [12]:
wavepower=wavepower_raw[['Year','Unnamed: 6']]

In [13]:
wavepower['Year']=wavepower['Year'].astype(int)

/var/folders/gg/6wnjn6cd1qlfst8mkw4g4xqc0000gn/T/ipykernel_24444/124231394.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wavepower['Year']=wavepower['Year'].astype(int)


In [14]:
wavepower=wavepower.rename(columns={'Unnamed: 6':'wave_power'})

In [15]:
wavepower_sst=wavepower.merge(seatemps, on="Year")
wavepower_sst['year']=pd.to_datetime(wavepower_sst['Year'], format="%Y")

In [16]:
wavepower_sst['wave_power_sn']=wavepower_sst['wave_power']/100000

#### wave power and temperature data together to see in datawrapper

In [17]:
wp_sst_dw=wavepower_sst[['year','wave_power_sn','Annual anomaly']]

In [18]:
dw.add_data(chart_id='lZ6uH',data=wp_sst_dw)

<Response [204]>

In [19]:
## This is the link to the pacific sea surface temp monthly data 
# relying on the google sheet but holding on the link 
# equatorial_pacific_sst_monthly = "https://www.cpc.ncep.noaa.gov/data/indices/sstoi.indices"

In [20]:
pt_processed=pacifictemps[['year','month','NINO4']]

In [21]:
pt_processed.rename(columns={'NINO4':'temp_nino4region'})
pt_processed['year_month']=pt_processed['month'] + "-" + pt_processed['year']

/var/folders/gg/6wnjn6cd1qlfst8mkw4g4xqc0000gn/T/ipykernel_24444/861877567.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pt_processed['year_month']=pt_processed['month'] + "-" + pt_processed['year']


In [22]:
pt_processed['year_month']=pd.to_datetime(pt_processed['year_month'], format="%m-%Y")

/var/folders/gg/6wnjn6cd1qlfst8mkw4g4xqc0000gn/T/ipykernel_24444/1265695988.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pt_processed['year_month']=pd.to_datetime(pt_processed['year_month'], format="%m-%Y")


In [23]:
grouped_df=pt_processed.groupby(['year','month','year_month'])

In [24]:
pt_processed['month']=pt_processed['month'].astype(int)

/var/folders/gg/6wnjn6cd1qlfst8mkw4g4xqc0000gn/T/ipykernel_24444/2930499555.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pt_processed['month']=pt_processed['month'].astype(int)


### trying to make a heatmap 

In [25]:
heatmap = (
    alt.Chart(pt_processed)
    .mark_rect()
    .encode(
        x=alt.X(
            "year:O",
            axis=alt.Axis(title=""),
        ),
        y=alt.Y(
            "month:O",
            axis=alt.Axis(title=""),
        ),
        color=alt.Color(
            ("NINO4"),
            title="",
            scale=alt.Scale(scheme="blueorange", reverse=True),
            legend=alt.Legend(type="gradient"),
        ),
    )
).properties(
    width=380,
    height=300,
    title=f" ",
)
heatmap.display()

alt.Chart(...)

In [26]:
%load_ext lab_black

In [27]:
seatemps
seatemps["year"] = pd.to_datetime(seatemps["Year"], format="%Y")

In [28]:
seatemps["y"] = 2

In [29]:
alt.Chart(seatemps).mark_bar().encode(
    x=alt.X("year", stack="normalize"),
    y=alt.Y("y", axis=alt.Axis(labels=False, ticks=False), title=""),
    color=alt.Color(
        ("Annual anomaly"), scale=alt.Scale(scheme="redblue", reverse=True)
    ),
).properties(width=800, height=100)

alt.Chart(...)

In [30]:
alt.Chart(seatemps).mark_rule().encode(
    x=alt.X("year", stack="normalize"),
    color=alt.Color(
        ("Annual anomaly"), scale=alt.Scale(scheme="redblue", reverse=True)
    ),
).properties(height=100, width=800)

alt.Chart(...)

In [31]:
alt.Chart(wp_sst_dw).mark_bar().encode(
    x=alt.X("year"),
    y=alt.Y("wave_power_sn"),
    color=alt.Color(
        ("Annual anomaly"), scale=alt.Scale(scheme="blueorange", reverse=False)
    ),
)

alt.Chart(...)

In [45]:
alt.Chart(wp_sst_dw).mark_trail().encode(
    x=alt.X("year"), y=alt.Y("wave_power_sn"), size=alt.Size("Annual anomaly")
)

alt.Chart(...)

In [51]:
wavepower["type"] = "wavepower"

In [65]:
wavepower_reframe = wavepower.rename(columns={"wave_power": "measure"})

In [80]:
wavepower

,Year,wave_power,type
0,1994,285607.95,wavepower
1,1995,280691.55,wavepower
2,1996,263888.53,wavepower
3,1997,292803.58,wavepower
4,1998,314412.76,wavepower
5,1999,287015.76,wavepower
6,2000,273477.50,wavepower
7,2001,270814.44,wavepower
8,2002,269126.92,wavepower
9,2003,271387.98,wavepower


In [61]:
seatemps = seatemps[seatemps["Year"] > 1993]

In [62]:
seatemps["type"] = "seatemp"

In [ ]:
seatemps_reframe = seatemps[["year", "Annual anomaly", "type"]].rename(
    columns={"Annual anomaly": "measure"}
)

In [73]:
sst_wp_rows = pd.concat([wavepower_reframe, seatempesreframe])

In [76]:
alt.Chart(sst_wp_rows).mark_area().encode(x="year", y="measure", row="type").properties(
    height=100
)

ValueError: year encoding field is specified without a type; the type cannot be inferred because it does not match any column in the data.

alt.Chart(...)

In [74]:
sst_wp_rows

,Year,measure,type
0,1994,285607.950,wavepower
1,1995,280691.550,wavepower
2,1996,263888.530,wavepower
3,1997,292803.580,wavepower
4,1998,314412.760,wavepower
5,1999,287015.760,wavepower
6,2000,273477.500,wavepower
7,2001,270814.440,wavepower
8,2002,269126.920,wavepower
9,2003,271387.980,wavepower
